In [5]:
from datetime import date
import pandas as pd
import numpy as np
import requests

today = date.today().strftime("%Y-%m-%d")

In [ ]:
url = f'https://dadosabertos.camara.leg.br/api/v2/deputados?siglaUf=AL&dataInicio=2019-01-31&dataFim={today}&ordem=ASC&ordenarPor=nome'
response = requests.get(url)
data = response.json()
dados = data['dados']
df_dep_init = pd.DataFrame(dados)
df_dep_init = df_dep_init[['id', 'nome', 'siglaPartido', 'siglaUf']]
df_dep_init['nome'] = df_dep_init['nome'].apply(lambda x: x.upper())
df_dep_init = df_dep_init.drop_duplicates(subset=['nome'])

In [83]:
temp_df = []

for dep in df_dep_init["id"]:
    response = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/deputados/{dep}')
    data = response.json()
    data = data['dados']["ultimoStatus"]
    temp_df.append(data)

df_dep = pd.DataFrame(temp_df)
df_dep = df_dep[["id","nome","siglaPartido","siglaUf","email","situacao","condicaoEleitoral"]]
    

In [ ]:
temp_df = []
k = 1
for id_dep in df_dep["id"]:
    while True:
        url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}/despesas?ano=2022&ano=2021&ano=2020&ano=2019&mes=12&mes=11&mes=10&mes=9&mes=8&mes=7&mes=6&mes=5&mes=4&mes=3&mes=2&mes=1&pagina={k}&itens=100&ordem=ASC&ordenarPor=ano'
        print(url)
        response = requests.get(url)
        data = response.json()
        data = data['dados']
        if len(data) == 0:
            break
        
        for i in range(len(data)):
            a = data[i]
            a['id'] = id_dep
            temp_df.append(a)
        
        k += 1

df_contas = pd.DataFrame(temp_df)

In [56]:
temp_df = []
k = 1
for index, id_dep in enumerate(df_dep["id"]):
    while True:
        url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}/discursos?dataInicio=2019-01-31&dataFim={today}&ordenarPor=dataHoraInicio&ordem=ASC&itens=100&pagina={k}'
        response = requests.get(url)
        data = response.json()
        data = data['dados']
        
        if len(data) == 0:
            break
        
        for i in range(len(data)):
            a = data[i]
            a['id'] = id_dep
            temp_df.append(a)
        
        k += 1

df_discursos = pd.DataFrame(temp_df)
df_discursos = pd.concat([df_discursos.drop(['faseEvento'],axis=1),df_discursos['faseEvento'].apply(pd.Series)], axis=1)

In [24]:
temp_df = []
k = 1
while True:
    url = f'https://dadosabertos.camara.leg.br/api/v2/votacoes?dataInicio=2022-01-01&dataFim={today}&pagina={k}&itens=200&ordem=DESC&ordenarPor=dataHoraRegistro'
    response = requests.get(url)
    data = response.json()
    data = data["dados"]
    
    if len(data) == 0:
        break
    
    temp_df.append(data)
    k += 1

df_votacoes = pd.DataFrame(temp_df)


In [100]:
temp_df = []
rows = df_votacoes.shape[0]
cols = df_votacoes.shape[1]

for i in range(0,rows):
    for j in range(0,cols):
        data = df_votacoes[j][i]
        if data != None:
            temp_df.append(data)

df_votacoes_atl = pd.DataFrame(temp_df)

In [ ]:
temp_df = []

for id in df_votacoes_atl["id"]:
    response = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/votacoes/{id}/votos')
    data = response.json()
    data = data['dados']
    if len(data) > 0:
        for i in range(len(data)):
            a = data[i]
            a['idVoto'] = id
            temp_df.append(a)

df_votos = pd.DataFrame(temp_df)
df_votos = pd.concat([df_votos.drop(['deputado_'],axis=1),df_votos['deputado_'].apply(pd.Series)], axis=1)

In [6]:
#df_dep_init.to_csv('deputados_al.csv', index=False, encoding='utf-8')
#df_votacoes_atl.to_csv('votacoes_atl.csv', index=False, encoding='utf-8')
#df_votos.to_csv('votos.csv', index=False, encoding='utf-8')
#df_contas.to_csv('contas.csv', index=False, sep=',', decimal='.', encoding='utf-8')
#df_discursos.to_csv('discursos.csv', index=False, sep=',', decimal='.', encoding='utf-8')